In [1]:
using Pkg;
using Statistics;

In [3]:
targets = convert(AbstractArray{Bool,2},[1 1 0 0 1 1 0 1 0 0 0; 0 0 1 0 0 0 1 0 1 0 1;0 0 0 1 0 0 0 0 0 1 0])
outputs = convert(AbstractArray{Bool,2},[1 0 0 0 0 1 0 1 1 0 0; 0 1 1 0 0 0 0 0 0 0 1;0 0 0 1 1 0 1 0 0 1 0])

(accuracy,precision, recall,fScore, mcm) = multiclassConfusionMatrix(outputs,targets)

(0.6363636363636364, 0.6388888888888888, 0.7000000000000001, 0.6680497925311203, [3.0 1.0 1.0; 1.0 2.0 1.0; 0.0 0.0 2.0])

In [2]:
function multiclassConfusionMatrix(outputs::AbstractArray{Bool,2}, targets::AbstractArray{Bool,2}; weighted::Bool=true)
    
    targets = targets'
    outputs = outputs'

    num_columns = size(targets,2)
    confusion_matrix= zeros(num_columns,num_columns)
  
      #confusion matrix
    for x in 1:num_columns
        for y in 1:num_columns
            for c in 1:(size(outputs,1))
                if outputs[c,x] == true
                    if outputs[c,x] == targets[c,y]
                        confusion_matrix[y,x] =confusion_matrix[y,x] + 1
                    end
                end
            end  
        end
    end

    mcm = confusion_matrix

    TP = zeros(size(mcm,1))
    precision = zeros(size(mcm,1))
    recall = zeros(size(mcm,1))

    for index in 1:size(mcm,1)
        TP[index] = mcm[index,index]
        STrow = sum(mcm[index,:])
        STcolumn = sum(mcm[:,index])
        precision[index] = TP[index]/STcolumn
        recall[index] = TP[index]/STrow
    end

    model_accuracy = sum(TP)/sum(mcm)
    model_precision = mean(precision)
    model_recall = mean(recall)
    model_fScore = (2*model_precision*model_recall)/(model_precision+model_recall)

    return (model_accuracy,model_precision, model_recall,model_fScore, mcm)
end


multiclassConfusionMatrix (generic function with 1 method)

In [ ]:
function multiclassConfusionMatrix(outputs::AbstractArray{Bool,2}, targets::AbstractArray{Bool,2};metric::Symbol=:all)
    print("hola")
    (model_accuracy,model_precision, model_recall,model_fScore, mcm) = multiclassConfusionMatrix(convert(AbstractArray{Bool,2},outputs),convert(AbstractArray{Bool,2},targets))

    if (metric == :all)
        return (model_accuracy,model_precision, model_recall,model_fScore, mcm)
    elseif (metric == :af)
        return (model_accuracy,model_fScore)
    elseif (metric == :pr)
        return (model_accuracy,model_fScore)
    elseif (metric == :afm)
        return (model_accuracy,model_fScore)
    elseif (metric == :m)
        return (model_accuracy,model_fScore)
    else
        return (model_accuracy,model_precision, model_recall,model_fScore, mcm)
    end
end


function confusionMatrix(outputs::AbstractArray{Bool,1}, targets::AbstractArray{Bool,1})

    @assert(length(outputs)==length(targets))

    TN = sum(.!outputs .& .!targets);
    FN = sum(.!outputs .& targets);
    TP = sum(outputs .&  targets)
    FP = sum(outputs .& .!targets)
    
    confMatrix = [TN FP; FN TP]
    
    #Metrics
    
    if ((TN+FN+TP+FP)==0)
        accuracy = 0
        error_rate = 0
    else
        accuracy = (TN+TP)/(TN+FN+TP+FP)
        error_rate = (FN+FP)/(TN+FN+TP+FP)
    end

    if (TN==length(outputs))
        recall = 1 #Sensitivity
        precision = 1 #Positive predicitive value
        specificity = TN/(FP+TN)
        negative_predicitve_value = TN/(TN+FN)
    elseif(TP==length(outputs))
        recall = TN/(FN+TP)
        precision = TP/(TP+FP)
        specificity = 1
        negative_predicitve_value = 1
    elseif(FN+TP == 0)
        recall = 0
        precision = TP/(TP+FP)
        specificity = TN/(FP+TN)
        negative_predicitve_value = TN/(TN+FN)
    elseif(TP+FP == 0)
        recall = TN/(FN+TP)
        precision = 0
        specificity = TN/(FP+TN)
        negative_predicitve_value = TN/(TN+FN)
    elseif(FP+TN == 0)
        recall = TN/(FN+TP)
        precision = TP/(TP+FP)
        specificity = 0
        negative_predicitve_value = TN/(TN+FN)
    elseif(TN+FN == 0)
        recall = TN/(FN+TP)
        precision = TP/(TP+FP)
        specificity = TN/(FP+TN)
        negative_predicitve_value = 0
    else
        recall = TN/(FN+TP)
        precision = TP/(TP+FP)
        specificity = TN/(FP+TN)
        negative_predicitve_value = TN/(TN+FN)
    end
    (precision==0 && recall==0) ? fscore = 0 : fscore = (2*recall*precision)/(recall+precision)
      
    return (confMatrix,accuracy,error_rate,recall,specificity,precision,negative_predicitve_value,fscore);
end